In [ ]:
##roBERTa

In [ ]:
!pip install transformers torch

In [ ]:
import pandas as pd

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [ ]:
import torch

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Encoding labels ('positive', 'negative', 'neutral') to numerical values

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
df['class'] = label_encoder.fit_transform(df['class'])

In [ ]:
# Tokenization and Data Preparation (using cleaned text)

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
encoded_data = tokenizer.batch_encode_plus(

In [ ]:
    df['cleaned_text'].tolist(),

In [ ]:
    add_special_tokens=True,

In [ ]:
    return_attention_mask=True,

In [ ]:
    padding='max_length',

In [ ]:
    max_length=128,  # Adjust if your sentences are longer

In [ ]:
    truncation=True,

In [ ]:
    return_tensors='pt'

In [ ]:
)

In [ ]:
input_ids = encoded_data['input_ids']

In [ ]:
attention_masks = encoded_data['attention_mask']

In [ ]:
labels = torch.tensor(df['label'].tolist())

In [ ]:
# Split into training and testing sets

In [ ]:
train_inputs, test_inputs, train_labels, test_labels = train_test_split(

In [ ]:
    input_ids, labels, random_state=35476648, test_size=0.2

In [ ]:
)

In [ ]:
train_masks, test_masks, _, _ = train_test_split(

In [ ]:
    attention_masks, input_ids, random_state=35476648, test_size=0.2

In [ ]:
)

In [ ]:
# Create DataLoader

In [ ]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)

In [ ]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)

In [ ]:
test_dataloader = DataLoader(test_data, batch_size=32)

In [ ]:
# Load Pre-trained Model

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3) # 3 labels for positive, negative, neutral

In [ ]:
# Enable GPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model.to(device)

In [ ]:
# Fine-tune (Optimizer, Loss Function, Training Loop)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
for epoch in range(3):  # Adjust number of epochs

In [ ]:
    model.train()

In [ ]:
    for batch in train_dataloader:

In [ ]:
        batch = tuple(t.to(device) for t in batch)

In [ ]:
        b_input_ids, b_input_mask, b_labels = batch

In [ ]:
        optimizer.zero_grad()

In [ ]:
        outputs = model(b_input_ids, attention_mask=b_input_mask)  # RoBERTa doesn't use token_type_ids

In [ ]:
        loss = loss_fn(outputs.logits, b_labels)

In [ ]:
        loss.backward()